# check oof df

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)


In [3]:
output_dir = os.path.join("/kaggle", "working", "_oof")
# exp_name = "debug"
exp_name = "exp012_targetdownsample_epoch10"
folds = [0, 1, 2, 3, 4]
# folds = [0]
# oof_df_path = os.path.join(output_dir, exp_name,"oof_df.parquet")
# oof_df = pd.read_parquet(oof_df_path)
print("load oof df")
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    df = pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


load oof df
fold 0


fold 1
fold 2
fold 3
fold 4


In [6]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

# 1step 0.5secで30minなら60*30=1800step
def postprocess_downsample_fn(df, N=11, maxpool_kernel_size=11, maxpool_stride=1):
    df = df.copy()
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(maxpool_kernel_size, stride=maxpool_stride, padding=maxpool_padding)
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df

def postprocess_downsample_notmaxpool(df, N=11):
    df = df.copy()
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [7]:
# df = postprocess_fn(oof_df, N=1200)
# sub_df = make_submission_df(df, threshold=0.2)


In [8]:
# data_num = 20000
# for _ in range(5):
#     random_num = np.random.randint(0, len(df)-data_num)
#     print(random_num)
#     plt.figure(figsize=(20, 5))
#     plt.subplot(3, 1, 1)
#     plt.plot(df["step"].iloc[random_num:random_num+data_num], df["event_pred"].iloc[random_num:random_num+data_num], label="class_pred")
#     plt.subplot(3, 1, 2)
#     plt.plot(df["step"].iloc[random_num:random_num+data_num], df["event"].iloc[random_num:random_num+data_num], label="event_pred")
#     plt.subplot(3, 1, 3)
#     plt.plot(df["step"].iloc[random_num:random_num+data_num], df["class_pred"].iloc[random_num:random_num+data_num], label="class_pred")


In [9]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [10]:
import sys
sys.path.append("/kaggle/src/dss_utils")

from dss_metrics import score


In [11]:
event_df = train_event_df[train_event_df["step"].notnull()].copy()


In [12]:
average_size = 31
max_kernel_size = 31
df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
sub_df = make_submission_df(df, threshold=0.5)


In [13]:
sub_df


,series_id,step,event_pred,event,score
328326,03d92c9f6f8a,5766.0,-0.731621,onset,0.731621
328967,03d92c9f6f8a,6407.0,0.924641,wakeup,0.924641
329760,03d92c9f6f8a,7200.0,0.969344,wakeup,0.969344
345600,03d92c9f6f8a,23040.0,-1.744578,onset,1.000000
346204,03d92c9f6f8a,23644.0,0.927520,wakeup,0.927520
...,...,...,...,...,...
81208080,fb223ed2278c,75600.0,0.971639,wakeup,0.971639
81223919,fb223ed2278c,91439.0,-0.968528,onset,0.968528
81224211,fb223ed2278c,143571.0,-0.866064,onset,0.866064
81224776,fb223ed2278c,144136.0,0.888754,wakeup,0.888754


In [14]:
score(event_df, sub_df)


0.6278161180940313

In [15]:
%%time
max_kernel_size = 31
tmp_score = 0
for average_size_ in range(11, 131, 10):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size)
    df = postprocess_downsample_fn(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_


average_size 11 max_kernel_size 31


: 

In [ ]:
tmp_score = 0
max_kernel_size_ = max_kernel_size
for max_kernel_size in range(11, 31, 10):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(event_df, sub_df)
    print(score_)
    # if tmp_score > score_:
    #     break
    tmp_score = score_
    max_kernel_size_ = max_kernel_size


average_size 21 max_kernel_size 11
0.6245531119003294
average_size 21 max_kernel_size 21
0.6213410307109896


In [ ]:
max_kernel_size = 3
df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
sub_df = make_submission_df(df, threshold=0.1)
print(len(sub_df))
if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(event_df, sub_df)
print(score_)


13866
0.6261576023828178


In [ ]:
# print("average_size", average_size, "max_kernel_size", max_kernel_size_)
# df = postprocess_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size_)
# sub_df = make_submission_df(df, threshold=0.01)
# score_ = score(event_df, sub_df)
# print(score_)
